# **Web Scraping Assignment 3 (Internship_23, Konatala Mohit, ID_34)**

In [112]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=2b908f4eec68531c590eeb8cff057b95628cc4115bbd374b6548784d4dfbe3ad
  Stored in directory: c:\users\kumari\appdata\local\pip\cache\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [1]:
# installing required libraries
import selenium 
from selenium import webdriver
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common import exceptions 
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import wget

Q1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.

In [31]:
# calling chrome driver
driver=webdriver.Chrome('chromedriver.exe')
# getting the url
url='https://www.amazon.in'
driver.get(url)
#enter the product name
input_str=input('Enter the product name that you wish to search for :')
#entering the keys
search_box=driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
search_box.send_keys(input_str)
# pressing enter to load the page
search_box.send_keys(Keys.ENTER)

Enter the product name that you wish to search for :laptop


Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search
results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [32]:
#defining function for fetching product url
product_url=[]
def attach(x,y):
    for i in x:
        y.append(i.text)

def url_fun(v,b):
    for j in range(0,len(v)):
        b.append(v[j].get_attribute('href'))

In [33]:
# fetching first 3 pages products urls
for i in range(0,3):
    try:
        getting_product_url=driver.find_elements_by_xpath('//a[@class="a-link-normal s-link-style a-text-normal"]')
        url_fun(getting_product_url, product_url)
    except StaleElementReferenceException:
        pass
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, 'Next'))).click()
        time.sleep(5)
    except exceptions.NoSuchElementException:
        break

In [55]:
# scraping the required data from each product url
name_list=[]
exchange_list=[]
price_list=[]
delivery_list=[]
stock_list=[]
brand_list=[]
y='-'
for i in range(0,len(product_url)):
    driver.get(product_url[i])
    time.sleep(3)
    try:
        product_name = driver.find_element_by_xpath('//div[@class="a-section a-spacing-none"]/h1/span')
        name_list.append(product_name.text)
    except:
        #if at all there is no data to scrape
        name_list.append(y)
    try:
        exchange=driver.find_element_by_xpath('//*[@id="maxBuyBackDiscountSection"]/span/span')
        exchange_list.append(exchange.text)
    except:
        exchange_list.append(y)
    try:
        price=driver.find_element_by_xpath('//span[@class="a-price a-text-price a-size-medium apexPriceToPay"]')
        price_list.append(price.text)
    except:
        price_list.append(y)
    try:
        delivery=driver.find_element_by_xpath('//*[@id="ddmDeliveryMessage"]/b')
        delivery_list.append(delivery.text)
    except:
        delivery_list.append(y)
    try:
        stock=driver.find_element_by_xpath('//div[@id="availability"]/span')
        stock_list.append(stock.text)
    except:
        stock_list.append(y)

for g in name_list:
    h=g.split(' ')
    brand_list.append(h[0])

In [56]:
#framing the data
scraped_details=pd.DataFrame(list(zip(brand_list,name_list,price_list,exchange_list,delivery_list,stock_list,product_url)))
scraped_details.columns=['Brand Name','Product','Price','Exchange Value upto','Delivery Date','Availablity','Product Url']
scraped_details.index+=1
pd.set_option('display.max_rows', None)
scraped_details

,Brand Name,Product,Price,Exchange Value upto,Delivery Date,Availablity,Product Url
1,Lenovo,Lenovo IdeaPad 3 10th Gen Intel Core i3 15.6 H...,"₹38,490.00","Up to 18,200.00 off",Feb 17 - 21,,https://www.amazon.in/gp/slredirect/picassoRed...
2,LG,LG Gram Intel Evo 11th Gen Core i7 17 inches U...,"₹94,999.00","Up to 18,200.00 off","Sunday, Jan 30",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
3,Lenovo,"Lenovo Chromebook 14e 14.0"" FHD Business Lapto...","₹20,990.00","Up to 15,650.00 off","Sunday, Jan 30",In stock.,https://www.amazon.in/Lenovo-Chromebook-Busine...
4,HP,HP Chromebook 14 Intel Celeron N4020-4GB SDRAM...,"₹27,390.00","Up to 18,200.00 off","Sunday, Jan 30",In stock.,https://www.amazon.in/HP-Chromebook-14a-na0003...
5,Dell,"Dell 15 (2021) Athlon Silver 3050U, 4GB, 256GB...","₹29,990.00","Up to 18,200.00 off",Jan 31 - Feb 2,Available to ship in 1-2 days.,https://www.amazon.in/Dell-Office21-Inspiron-3...
6,ASUS,ASUS ZenBook 14 UX433FA-A6106T 8th Gen Intel C...,"₹72,999.00","Up to 18,200.00 off","Wednesday, Feb 2",Only 1 left in stock.,https://www.amazon.in/ZenBook-UX433FA-A6106T-i...
7,Acer,Acer Swift 3 AMD Ryzen 5 4500U 14 inches Displ...,"₹59,990.00","Up to 18,200.00 off","Thursday, Feb 3",In stock.,https://www.amazon.in/Acer-14-inch-Display-Gra...
8,HP,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",-,-,"Wednesday, Feb 2",In stock.,https://www.amazon.in/HP-3-3250-Laptop-Windows...
9,Acer,Acer Aspire 3 AMD Athlon Silver 3050U Dual-cor...,"₹31,990.00","Up to 18,200.00 off","Thursday, Feb 3",In stock.,https://www.amazon.in/Acer-Aspire-Athlon-Dual-...
10,Dell,"Dell New 15 Intel i3-1115G4 Laptop, 8GB, 256GB...","₹40,690.00","Up to 18,200.00 off","Sunday, Jan 30",In stock.,https://www.amazon.in/Dell-i3-1115G4-Integrate...


In [57]:
scraped_details.to_csv('product_details.csv')

Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [3]:
#loading the driver and defining list of data that need to be searched in google images
driver=webdriver.Chrome('chromedriver.exe')
img_keys=['fruits','cars','Machine Learning', 'Guitar','Cakes']

In [4]:
#defining function to fetch the images of top 10 images 
def getting_img(z):
    url='https://images.google.com'
    driver.get(url)
    search_bar=driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
    search_bar.send_keys(z)
    search_bar.send_keys(Keys.ENTER)
    images_url=[]
    for i in range(1,11):
        try:
            images=driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div['+str(i)+']/a[1]').click()
            time.sleep(5)
            clear_pic=WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div/a/img"))).get_attribute("src")                                   
            images_url.append(clear_pic)
            time.sleep(2)
        except :
            continue
    
    x=1
    for j in images_url:
        driver.get(j)
        time.sleep(2)
        driver.save_screenshot(z+str(x)+'.png')
        x+=1

In [5]:
#creating a to call function for each string defined in img_keys
for y in img_keys:
    getting_img(y)

Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [5]:
#loading the driver and url page
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.flipkart.com'
driver.get(url)
driver.maximize_window()
time.sleep(3)
#closing the popup window
close_button=driver.find_element_by_xpath('/html/body/div[2]/div/div/button').click()

In [6]:
#entering desired input
phone_name=input("Enter the name of the phone to know the details :")
searchbox=driver.find_element_by_xpath('//div[@class="_3OO5Xc"]/input')
searchbox.send_keys(phone_name)
searchbox.send_keys(Keys.ENTER)

Enter the name of the phone to know the details :vivo


In [7]:
#fetching url of the products on 1st page
product_url_list=[]
product_url=driver.find_elements_by_xpath('//div[@class="_2kHMtA"]/a')
for i in product_url:
    product_url_list.append(i.get_attribute('href'))

price_list=[]
def attach(x,y):
    for i in x:
        y.append(i.text)
#fetching price details from 1st page
price=driver.find_elements_by_xpath('//div[@class="_30jeq3 _1_WHN1"]')
attach(price,price_list)
#fetching battery detail from 1st page
q=phone_name.lower()
battery_capacity_list=[]
#if entered is an iphone exception
if q[0:6]=='iphone':
    for r in price_list:
        battery_capacity_list.append('-')
else:
    try:
        battery_capacity=driver.find_elements_by_xpath('//div[@class="fMghEO"]/ul/li[4]')
        attach(battery_capacity,battery_capacity_list)
    except:
         battery_capacity_list.append('-')

# acquired all the other required data
display_size_list=[]
display_size=driver.find_elements_by_xpath('//div[@class="fMghEO"]/ul/li[2]')
attach(display_size,display_size_list)

cam_list=[]
primarycam_list=[]
secondarycam_list=[]
cam=driver.find_elements_by_xpath('//div[@class="fMghEO"]/ul/li[3]')
attach(cam,cam_list)
#filtering primary and secondary cam data
for i in cam_list:
    x=[]
    x=i.split('| ')
    try:
        primarycam_list.append(x[0])
    except:
        primarycam_list.append('-')
    try:    
        secondarycam_list.append(x[1])
    except:
        secondarycam_list.append('-')

memory_list=[]
ram_list=[]
rom_list=[]
storage=driver.find_elements_by_xpath('//div[@class="fMghEO"]/ul/li[1]')
attach(storage,memory_list)
#filtering memory data 
if q[0:6]=='iphone':
    for j in memory_list:
        rom_list.append(j)
        ram_list.append('-')
else:
    for i in memory_list:
        x=[]
        x=i.split('| ')
        try:
            ram_list.append(x[0])
        except:
            ram_list.append('-')
        try:    
            rom_list.append(x[1])
        except:
            rom_list.append('-') 

product=[]
name=[]
color=[]
description=driver.find_elements_by_xpath('//div[@class="_4rR01T"]')
attach(description,product)
#getting the colour data from the product name
for i in product:
    y=i.split('(')
    try:
        name.append(y[0])
    except:
        name.append('-')
    try:
        color.append(y[1])
    except:
        color.append('-')

colour=[]
for z in color:
    f=z.split(',')
    colour.append(f[0])
#getting brand name and smartphone name from the product name
brand_name_list=[]
smartphone_list=[]
for i in name:
    u=i.split(" ", 1)
    try:
        brand_name_list.append(u[0])
    except:
        brand_name_list.append('-')
    try:
        smartphone_list.append(u[1])
    except:
        smartphone_list.append('-')

df=pd.DataFrame(list(zip(brand_name_list,smartphone_list,colour,ram_list,rom_list,primarycam_list,secondarycam_list,display_size_list,battery_capacity_list,price_list,product_url_list)))
df.columns=['Brand Name','Smartphone','Colour','RAM','ROM','Primary Camera','Secondary Camera','Display','Battery capacity','Price','Product url']
df.index+=1
df

,Brand Name,Smartphone,Colour,RAM,ROM,Primary Camera,Secondary Camera,Display,Battery capacity,Price,Product url
1,vivo,V23 5G,Sunshine Gold,12 GB RAM,256 GB ROM,64MP + 8MP + 2MP,50MP + 8MP Dual Front Camera,16.36 cm (6.44 inch) Full HD+ Display,4200 mAh Lithium Battery,"₹34,990",https://www.flipkart.com/vivo-v23-5g-sunshine-...
2,vivo,V23 5G,Stardust Black,8 GB RAM,128 GB ROM,64MP + 8MP + 2MP,50MP + 8MP Dual Front Camera,16.36 cm (6.44 inch) Full HD+ Display,4200 mAh Lithium Battery,"₹29,990",https://www.flipkart.com/vivo-v23-5g-stardust-...
3,vivo,V23 5G,Stardust Black,12 GB RAM,256 GB ROM,64MP + 8MP + 2MP,50MP + 8MP Dual Front Camera,16.36 cm (6.44 inch) Full HD+ Display,4200 mAh Lithium Battery,"₹34,990",https://www.flipkart.com/vivo-v23-5g-stardust-...
4,vivo,V23 5G,Sunshine Gold,8 GB RAM,128 GB ROM,64MP + 8MP + 2MP,50MP + 8MP Dual Front Camera,16.36 cm (6.44 inch) Full HD+ Display,4200 mAh Lithium Battery,"₹29,990",https://www.flipkart.com/vivo-v23-5g-sunshine-...
5,vivo,Y12G,Phantom Black,3 GB RAM,32 GB ROM,13MP + 2MP,8MP Front Camera,16.54 cm (6.51 inch) HD+ Display,5000 mAh Li-ion Battery,"₹10,990",https://www.flipkart.com/vivo-y12g-phantom-bla...
6,vivo,Y12G,Glacier Blue,3 GB RAM,32 GB ROM,13MP + 2MP,8MP Front Camera,16.54 cm (6.51 inch) HD+ Display,5000 mAh Li-ion Battery,"₹10,990",https://www.flipkart.com/vivo-y12g-glacier-blu...
7,vivo,Y20G 2021,Obsidian Black,4 GB RAM,64 GB ROM,13MP + 2MP + 2MP,8MP Front Camera,16.54 cm (6.51 inch) HD+ Display,5000 mAh Li-ion Battery,"₹13,990",https://www.flipkart.com/vivo-y20g-2021-obsidi...
8,vivo,Y20T,Purist Blue,6 GB RAM,64 GB ROM,13MP + 2MP + 2MP,8MP Front Camera,16.54 cm (6.51 inch) HD+ Display,5000 mAh Lithium Battery,"₹15,490",https://www.flipkart.com/vivo-y20t-purist-blue...
9,vivo,Y20T,Obsidian Black,6 GB RAM,64 GB ROM,13MP + 2MP + 2MP,8MP Front Camera,16.54 cm (6.51 inch) HD+ Display,5000 mAh Lithium Battery,"₹15,490",https://www.flipkart.com/vivo-y20t-obsidian-bl...
10,vivo,Y12G,Phantom Black,3 GB RAM,64 GB ROM,13MP + 2MP,8MP Front Camera,16.54 cm (6.51 inch) HD+ Display,5000 mAh Li-ion Battery,"₹11,990",https://www.flipkart.com/vivo-y12g-phantom-bla...


In [33]:
df.to_csv('smartphone_details.csv')

Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.

In [34]:
#loading the driver and url
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.google.com/maps'
driver.get(url)
driver.maximize_window()
time.sleep(3)

In [35]:
#entering the desired city as input
city=input("Enter the City to fetch Geospatial Coordinates : ")
c=city.lower()
#locating the search bar and sending the entered input to the search bar
searchbox=driver.find_element_by_xpath('//div[@id="gs_lc50"]/input')
searchbox.send_keys(c)
searchbox.send_keys(Keys.ENTER)
time.sleep(2)
#getting and url of the page after search to get latitude and longitude of the city
present_url=driver.current_url
data=present_url.split('/')
cords=data[6]
cords=cords[1:]
cords_vals=cords.split(',')
lat=cords_vals[0]
lon=cords_vals[1]
#printing the values
print('The Coordinates of ',c,' city\n','Latitude : ',lat,'\n','Longitude : ',lon)

Enter the City to fetch Geospatial Coordinates : Jaipur
The Coordinates of  jaipur  city
 Latitude :  26.8851417 
 Longitude :  75.6504714


Q6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)
from trak.in.

In [133]:
#loading the driver and page url 
driver=webdriver.Chrome('chromedriver.exe')
url='https://trak.in'
driver.get(url)
driver.maximize_window()
time.sleep(2)
#fetching url of funding deals from the website
funding_deals=driver.find_element_by_xpath('//ul[@class="main-menu menu clearfix bsm-initialized"]/li[9]/a')
url1=funding_deals.get_attribute('href')
driver.get(url1)
time.sleep(3)

In [134]:
#defining empty list
date_list=[]
startup_list=[]
industry_list=[]
subv_list=[]
city_list=[]
invn_list=[]
invt_list=[]
amt_list=[]
jan=54
feb=55
mar=56
#defining function to fetch the details from the tables of particular month
def month_deal(x):
    for i in range(1,11):
        try:
            j=2
            date=driver.find_element_by_xpath('//table[@id="tablepress-'+str(x)+'"]/tbody/tr['+str(i)+']/td['+str(j)+']').text
            date_list.append(date)
            j+=1
            startup=driver.find_element_by_xpath('//table[@id="tablepress-'+str(x)+'"]/tbody/tr['+str(i)+']/td['+str(j)+']').text
            startup_list.append(startup)
            j+=1
            industry=driver.find_element_by_xpath('//table[@id="tablepress-'+str(x)+'"]/tbody/tr['+str(i)+']/td['+str(j)+']').text
            industry_list.append(industry)
            j+=1
            subv=driver.find_element_by_xpath('//table[@id="tablepress-'+str(x)+'"]/tbody/tr['+str(i)+']/td['+str(j)+']').text
            subv_list.append(subv)
            j+=1
            city=driver.find_element_by_xpath('//table[@id="tablepress-'+str(x)+'"]/tbody/tr['+str(i)+']/td['+str(j)+']').text
            city_list.append(city)
            j+=1
            invn=driver.find_element_by_xpath('//table[@id="tablepress-'+str(x)+'"]/tbody/tr['+str(i)+']/td['+str(j)+']').text
            invn_list.append(invn)
            j+=1
            invt=driver.find_element_by_xpath('//table[@id="tablepress-'+str(x)+'"]/tbody/tr['+str(i)+']/td['+str(j)+']').text
            invt_list.append(invt)
            j+=1
            amt=driver.find_element_by_xpath('//table[@id="tablepress-'+str(x)+'"]/tbody/tr['+str(i)+']/td['+str(j)+']').text
            amt_list.append(amt)
        except NoSuchElementException:
            break
time.sleep(1)
#fetching details of particular month 
month_deal(jan)
month_deal(feb)
month_deal(mar)
#framing the data
month_deal_df=pd.DataFrame(list(zip(date_list,startup_list,industry_list,subv_list,city_list,invn_list,invt_list,amt_list)))
month_deal_df.columns=['Date','Startup Name','Industry','Sub-Vertical','City','Investor Name','Investment Type','Amount (in USD)']
month_deal_df.index+=1
month_deal_df

,Date,Startup Name,Industry,Sub-Vertical,City,Investor Name,Investment Type,Amount (in USD)
1,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
2,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
3,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
4,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
5,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
6,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
7,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
8,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
9,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
10,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


Q7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [58]:
#loading the driver and page url
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.digit.in'
driver.get(url)
driver.maximize_window()
time.sleep(2)
#clicking on laptops
laptops=driver.find_element_by_xpath('/html/body/div[2]/div/div[4]/ul/li[3]/a').click()
time.sleep(2)
#clicking on top programmer laptops
prog_laptops=driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[2]/ul/li[3]/a').click()
time.sleep(2)
#defining empty list and function required
name_list=[]
price_list=[]
os_list=[]
display_list=[]
processor_list=[]
memory_list=[]
rom_list=[]
ram_list=[]
weight_list=[]
dimension_list=[]
graphics_list=[]
def attach(x,y):
    for i in x:
        y.append(i.text)
# finding details of elements that are required        
name=driver.find_elements_by_xpath('//div[@data-cat="Top_Ten_En_Scroll"]/a')
price=driver.find_elements_by_xpath('//td[@class="smprice"]')
os=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[3]/td[3]')
display=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[4]/td[3]')
processor=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[5]/td[3]')
memory=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[6]/td[3]')
weight=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
dimension=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]')
graphics=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[9]/td[3]')
# appending the acquired elements to the empty list
attach(name,name_list)
attach(price,price_list)
attach(os,os_list)
attach(display,display_list)
attach(processor,processor_list)
attach(memory,memory_list)
attach(weight,weight_list)
attach(dimension,dimension_list)
attach(graphics,graphics_list)
# splitting memory into ram and rom 
for j in memory_list:
    k=j.split('/')
    str(k)
    rom_list.append(k[0])
    ram_list.append(k[1])
#framing the data  
top_prog_laptops=pd.DataFrame(list(zip(name_list,price_list,os_list,display_list,processor_list,rom_list,ram_list,weight_list,dimension_list,graphics_list)))
top_prog_laptops.columns=['Name','Price','OS','Display','Processor','ROM','RAM','Weight','Dimension','Graphics Card']
top_prog_laptops.index+=1
top_prog_laptops

,Name,Price,OS,Display,Processor,ROM,RAM,Weight,Dimension,Graphics Card
1,FUJITSU UH-X 2-IN-1,"₹ 107,990",Windows 10,"13.3"" (1922 x 1080)",11th Gen Intel Tiger Lake Core i7-1165G7 | 2.8...,1 TB SSD,16 GBGB DDR4,997,30.9 x 21.5 x 1.7,Intel Iris Xe Integrated
2,ASUS ROG ZEPHYRUS G14,"₹ 83,990",Windows 10 Home,"14"" (2560 x 1440)",AMD Ryzen 7 Octa Core-4800HS | 2.9 GHz,1 TB SSD,8 GBGB DDR4,1.60,324 x 222 x 17.9,NVIDIA GeForce GTX 1650
3,LENOVO YOGA SLIM 7I,"₹ 108,999",Windows 10 Home,"13.3"" (2560 x 1600)",11th Gen Intel® Core™ i7-1165G7 | 2.80 GHz,1 TB SSD,16 GBGB DDR4,1.36 Kg,32.1 x 20.8 x 1.5,Intel® Iris® Xe
4,MACBOOK PRO 13-INCH,"₹ 119,900",NA,"13.3"" (2560 x 1600)",Quad-core 8th-gen Intel Core i5 | 4.1 GHz,256 GB SSD,8 GBGB DDR4,2.3,1.49 x 30.41 x 21.24,Intel UHD Graphics 620
5,ALIENWARE M15,"₹ 149,990",Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen R7-5800H | 4.4 GHz,512 GB SSD,16 GBGB DDR4,2.42,27.2 x 35.7 x 2.5,NVIDIA GeForce RTX 3060
6,APPLE MACBOOK AIR M1,"₹ 92,900",MacOS 10.14 Mojave,"13.3"" (2560 x 1600)",8-core CPU | NA,256 GB SSD,8 GBGB DDR4,2.08,32.4 x 7.3 x 23.1,Apple 7-core
7,REALME BOOK,"₹ 44,999",Windows 10,"14"" (2160 × 1440)",11th Gen Intel® Core™ i3-1115G4 | 3.0GHz,256 GB SSD,8 GBGB DDR4,1.38 Kg,307 x 228.96 x 14.9,Intel® UHD Graphics
8,ACER SWIFT 3,"₹ 58,757",Windows 10 Home,"14 inch"" (1920 x 1080)",Ryzen 5 4500U | 4.0Ghz,512 GB NVMe,8 GBGB LPDDR4,1.2,21.8 x 32.3 x 1.6 cm,AMD Radeon Graphics
9,DELL XPS 15,"₹ 98,200",Windows 10 Home,"15.6"" (3840 x 2400)",10th-generation Intel Core i9-10885H | NA,256 GB SSD,16 GBGB DDR4,2.05,344.72 x 230.14 x 18,NA
10,APPLE MACBOOK PRO 16-INCH,"₹ 199,900",Ios,"16"" (3072 X 1920)",Intel Core i9 8th Gen | NA,512 GB SSD,16 GBGB DDR3,1.83,349.3 x 240.7 x 15.5,AMD Radeon Pro 560X


Q8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [5]:
#loading the driver and getting the url
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.forbes.com'
driver.get(url)
driver.maximize_window()
time.sleep(3)
#clicking on the hamburger button
ham=driver.find_element_by_xpath('//button[@class="icon--hamburger"]').click()
#loading action chain to click on the worlds billionaire
action=ActionChains(driver)
billi=driver.find_element_by_xpath('//li[@data-title="Billionaires"]')
action.move_to_element(billi).perform()
wbilli=driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a').click()

In [6]:
#defining empty lists
name_list=[]
name_list1=[]
net_list=[]
age_list=[]
country_list=[]
source_list=[]
industry_list=[]
#creating a loop to obtain all pages billionaires data
for x in range(0,14):
    time.sleep(2)
# closing the details of top list data
    upper_arrow= WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[7]/div/span'))).click()
#fetching all the required data
    for i in range(1,28,2):
        for j in range(2,16):
            try:
                k=2
                name=driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div['+str(i)+']/div['+str(j)+']/div['+str(k)+']').text
                name_list.append(name)
                k+=1
                net=driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div['+str(i)+']/div['+str(j)+']/div['+str(k)+']').text
                net_list.append(net)
                k+=1
                age=driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div['+str(i)+']/div['+str(j)+']/div['+str(k)+']').text
                age_list.append(age)
                k+=1
                country=driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div['+str(i)+']/div['+str(j)+']/div['+str(k)+']').text
                country_list.append(country)
                k+=1
                source=driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div['+str(i)+']/div['+str(j)+']/div['+str(k)+']').text
                source_list.append(source)
                k+=1
                industry=driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div['+str(i)+']/div['+str(j)+']/div['+str(k)+']').text
                industry_list.append(industry)
            except NoSuchElementException:
                break
    name_list.pop()
    try:
        next_button=driver.find_element_by_xpath('//div[@class="next-page"]/i').click()
        time.sleep(1)
    except exceptions.ElementClickInterceptedException:
        break

In [13]:
#framing the data
billi_list=pd.DataFrame(list(zip(name_list,net_list,age_list,country_list,source_list,industry_list)))
billi_list.columns=['Name','Net Worth','Age','Country','Source','Industry']
billi_list.index+=1
billi_list

,Name,Net Worth,Age,Country,Source,Industry
1,Jeff Bezos,$177 B,57,United States,Amazon,Technology
2,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
3,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
4,Bill Gates,$124 B,65,United States,Microsoft,Technology
5,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...
2572,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2573,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2574,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2575,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video

In [2]:
#entering the url and loading driver and url
url=input('Enter the Youtube URL : ')
driver=webdriver.Chrome('chromedriver.exe')
driver.get(url)
driver.maximize_window()
time.sleep(3)
#defining empty list
time_list=[]
upvote_list=[]
comment_list=[]
#creating a loop to scroll down the page
for j in range(500,100000,500):
    i=0
    X=driver.execute_script("window.scrollTo("+str(i)+","+str(j)+");")
    i=j
    time.sleep(2)
#creating a loop to get the required details
for z in range(1,501):
    try:
        time=driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/ytd-comments/ytd-item-section-renderer/div[3]/ytd-comment-thread-renderer['+str(z)+']/ytd-comment-renderer/div[3]/div[2]/div[1]/div[2]/yt-formatted-string').text
        time_list.append(time)
        upvote=driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/ytd-comments/ytd-item-section-renderer/div[3]/ytd-comment-thread-renderer['+str(z)+']/ytd-comment-renderer/div[3]/div[2]/ytd-comment-action-buttons-renderer/div[1]/span[2]').text
        upvote_list.append(upvote)
        comment=driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/ytd-comments/ytd-item-section-renderer/div[3]/ytd-comment-thread-renderer['+str(z)+']/ytd-comment-renderer/div[3]/div[2]/ytd-expander/div/yt-formatted-string[2]').text
        comment_list.append(comment)
    except NoSuchElementException:
        break

Enter the Youtube URL : https://www.youtube.com/watch?v=STO4-8vkG0U


In [3]:
#framing the data
top_500_comments=pd.DataFrame(list(zip(comment_list,upvote_list,time_list)))
top_500_comments.columns=['Comment','Upvote','Time']
top_500_comments.index+=1
top_500_comments

,Comment,Upvote,Time
1,This aesthetic is made for Selena,10K,1 year ago
2,This era of selena deserved more recognition s...,10K,8 months ago
3,This is seriously one of her top best songs an...,7.3K,8 months ago
4,"This is literally her best song, music video, ...",1.1K,2 months ago
5,It's crazy how many people come back to this e...,687,1 month ago
...,...,...,...
496,I don't know why but I know that sel makes me ...,1,2 months ago
497,I would do everything to Look like Selena and ...,1,2 months ago (edited)
498,Esta canción era la mejor era para Selena Gómez,6,2 months ago
499,Director: so how beautiful do you want this vi...,140,2 years ago


Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews,
overall reviews, privates from price, dorms from price, facilities and property description

In [2]:
#loading the driver and url page
url='https://www.hostelworld.com/'
driver=webdriver.Chrome('chromedriver.exe')
driver.get(url)
driver.maximize_window()
time.sleep(3)
#clicking on the search button
search_button_click=driver.find_element_by_xpath('//div[@class="field body-2 location-input"]/div/input').click()
time.sleep(5)
#finding the search bar
search_buttons=driver.find_element_by_xpath('//input[@class="search-input"]')
search_buttons.send_keys('London')
time.sleep(15)
#selecting the list prediction of the search bar
choice=driver.find_element_by_xpath('//li[@tabindex="-1"][2]').click()
lets_go=driver.find_element_by_xpath('//button[@class="button primary large"]').click()

In [11]:
#defining empty list
header_list=[]
link_list=[]
price_list=[]
distance_list=[]
#creating a loop to fetch hostel name, url, distance from city centre and prices
for i in range(0,3):
    time.sleep(3)
    header_and_links=driver.find_elements_by_xpath('//h2[@class="title title-6"]/a')
    distance_from_centre=driver.find_elements_by_xpath('//span[@class="description"]')
    prices=driver.find_elements_by_xpath('//a[@class="prices"]/div')
    for j in header_and_links:
        header_list.append(j.text)
        link_list.append(j.get_attribute('href'))
    for k in distance_from_centre:
        distance_list.append(k.text)
    for l in prices:
        price_list.append(l.text)
    try:
        next=driver.find_element_by_xpath('//i[@class="core-icon icon-core-chevron-right"]').click()
    except exceptions.ElementClickInterceptedException:
        break
#releasing the values as it is non required string
price_list.pop(0)
#separating prices of private and dorm room
private_list=[]
dorm_list=[]
for q in range(0,len(price_list),2):
    private_list.append(price_list[q])
    q+=1
    dorm_list.append(price_list[q])
#removing all unrequired data from private and dorm prices
new_private_list=[]
new_dorm_list=[]
for w in range(0,len(private_list)):
    i=private_list[w]
    str(i)
    new_private_list.append(i[-6:])
    j=dorm_list[w]
    str(j)
    new_dorm_list.append(j[-6:])
#reframing the not available data in private list
private_from_price_list=[]
for i in new_private_list:
    if i=='ilable':
        private_from_price_list.append('Not Available')
    elif i[0]=='s':
        private_from_price_list.append('R'+str(i))
    else:
        private_from_price_list.append(i)
#reframing the not available data in dorm list      
dorm_from_price_list=[]
for i in new_dorm_list:
    if i=='ilable':
        dorm_from_price_list.append('Not Available')
    elif i[0]=='\n':
        dorm_from_price_list.append(i[1:])
    else:
        dorm_from_price_list.append(i)
#defining empty lists      
propert_des_list=[]
rating_list=[]
overall_r_list=[]
total_reviews_list=[]
facility_list=[]
# fetching property description,ratings,overall reviews, total reviews, facilities using url of individual hostel
for i in link_list:
    driver.get(i)
    time.sleep(3)
    try:
        more=driver.find_element_by_xpath('//a[@class="toggle-content"]').click()
    except:
        pass
    property_des=driver.find_element_by_xpath('//div[@style="-webkit-line-clamp: 12;"]')
    propert_des_list.append(property_des.text)
    rating=driver.find_element_by_xpath('//div[@class="rating rating-summary-container big clickable"]/div')
    rating_list.append(rating.text)
    try:
        overall_r=driver.find_element_by_xpath('//div[@class="rating rating-summary-container big clickable"]/div[2]/div')
        overall_r_list.append(overall_r.text)
    except:
        overall_r_list.append('-')
    total_reviews=driver.find_element_by_xpath('//div[@class="rating rating-summary-container big clickable"]/div/div[2]')
    total_reviews_list.append(total_reviews.text)
    fac=driver.find_element_by_xpath('//ul[@class="facilities"]')
    facility_list.append(fac.text)

In [12]:
#framing the data
Hostel_details=pd.DataFrame(list(zip(header_list,distance_list,rating_list,total_reviews_list,overall_r_list,private_from_price_list,dorm_from_price_list,facility_list,propert_des_list)))
Hostel_details.columns=['Hostel Name','Distance from city centre','Ratings','Total reviews','Overall review','Private Price','Dorm Price','Facilities','Property Description']
Hostel_details.index+=1
Hostel_details

,Hostel Name,Distance from city centre,Ratings,Total reviews,Overall review,Private Price,Dorm Price,Facilities,Property Description
1,Mornington Camden,Hostel - 4.1km from city centre,8.3,44 Total Reviews,Fabulous,Not Available,Rs3277,Linen Included Free City Maps Towels Included ...,Mornington Camden Hostel is a high class spaci...
2,Smart Russell Square Hostel,Hostel - 2.6km from city centre,6.7,9567 Total Reviews,Good,Not Available,Rs1205,Linen Included Free City Maps Free WiFi Free I...,Many Hostelworld guests who visit Smart Russel...
3,Smart Camden Inn Hostel,Hostel - 4.4km from city centre,8.5,2739 Total Reviews,Fabulous,Rs5197,Rs1733,Linen Included Free City Maps Free WiFi,"Located in the middle of Camden Town, we’re th..."
4,Selina Camden,Hostel - 5.5km from city centre,9.1,14 Total Reviews,Superb,Rs12901,Rs3548,Linen Included Towels Included Free WiFi,"Among underground music venues, innovative mus..."
5,Queen Elizabeth Chelsea,Hostel - 5.7km from city centre,7.5,3214 Total Reviews,Very Good,Rs4112,Rs1821,Linen Included Free City Maps Free WiFi Free I...,PLEASE NOTE WE ONLY ACCEPT GUESTS FROM THE AGE...
...,...,...,...,...,...,...,...,...,...
73,Green Rooms,Hotel - 10.8km from city centre,9.7,76 Total Reviews,Superb,Rs7844,Rs2477,Linen Included Free City Maps Towels Included ...,Green Rooms is a pioneering arts-led hotel and...
74,23 Matheson Road,Bed and Breakfast - 6.2km from city centre,No Rating\n0 Total Reviews,0 Total Reviews,-,Rs5025,Not Available,Towels Included,This pretty Victorian town house on a quiet st...
75,Park Hotel Essex,Hotel - 24.1km from city centre,8.0,109 Total Reviews,Fabulous,Rs11636,Not Available,Free Breakfast Free Parking,This Hotel is the right choice for visitors wh...
76,Cranbrook Hotel,Hotel - 14.8km from city centre,No Rating\n0 Total Reviews,0 Total Reviews,-,Rs4148,Not Available,Free Breakfast Free Parking,We are located about twenty minutes by tube fr...
